<h1 style="font-size:24px;">       机器学习大作业————肺部X光图像分类（是否为肺结核）</h1>

**任务类型**：基于sklearn中四种机器学习算法（KNN，SVM，决策树，随机森林），两种深度学习架构（resneXt50，VIT）的肺部X光二分类任务（是否为肺结核样本）

**数据来源**：https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset/data

**数据量**：有标签的数据共4200张图片

**训练设备**：4090D（24GB）



In [ ]:
import numpy  as np
import pandas as pd
from skimage.transform import resize
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import os
import matplotlib.image as mpimg  # Matplotlib库中的image模块，用于图像处理

展示部分数据

随机选择和显示正常胸部X光图像数据集中的样本，有助于检查和验证图像数据的质量和内容。

通过可视化样本图像，可以直观地了解数据集的情况，为后续的数据处理和模型训练做好准备。

In [ ]:
import random
# 设置图形大小，宽度为20，高度为20
plt.figure(figsize=(20,20))
# 指定正常胸部X光图像的文件夹路径
folder=r'TB_Chest_Radiography_Database/Normal'
# 随机选择并显示5张正常胸部X光图像
for i in range(5):
    file = random.choice(os.listdir(folder))
    image_path= os.path.join(folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [ ]:
#同上方法查看部分肺结核样本
import random
plt.figure(figsize=(20,20))
folder=r'TB_Chest_Radiography_Database/Tuberculosis'
for i in range(5):
    file = random.choice(os.listdir(folder))
    image_path= os.path.join(folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

机器学习部分，直接调包，无需自己搭模型

数据读入与预处理

In [ ]:
# 定义类别列表
Categories = ["Normal","Tuberculosis"]
# 初始化用于存储图像数据和平铺图像数据的列表
flat_data_arr = []
target_arr = []
datadir = "TB_Chest_Radiography_Database/"
# 遍历每个类别
for category in Categories:
    print(f"正在加载类别：{category}")
    path = os.path.join(datadir, category)
    for img in os.listdir(path):
        try:
            img_path = os.path.join(path, img)
              # 读取图像文件
            img_array = imread(img_path, plugin='imageio')
            # 调整图像大小为150x150，并保留3个颜色通道（RGB）
            img_resized = resize(img_array, (150, 150, 3), anti_aliasing=True)
              # 将调整大小后的图像展平并添加到数据列表中
            flat_data_arr.append(img_resized.flatten())
             # 将类别索引添加到目标列表中
            target_arr.append(Categories.index(category))
               # 捕捉并打印可能发生的异常
        except Exception as e:
            print("")
    print(f"已加载类别 {category}")
# 将平铺的图像数据和目标数据转换为NumPy数组
flat_data_arr = np.array(flat_data_arr)
target_arr = np.array(target_arr)
# 打印加载完成的消息
print("所有类别图片已加载完成。")

展示部分数据

In [ ]:
# 将平铺的图像数据和目标数据转换为NumPy数组
flat_data = np.array(flat_data_arr)
target = np.array(target_arr)
# 创建一个DataFrame，将平铺的图像数据存储在DataFrame中
df = pd.DataFrame(flat_data)
# 添加目标标签列
df["Target"] = target
df.head()


In [ ]:
X = df.drop('Target',axis=1)
X.head()

In [ ]:
Y = df[['Target']]
Y.head()

In [ ]:
from sklearn import svm
#导入支持向量机（Support Vector Machine，SVM）模型，用于分类任务。
from sklearn.neighbors import KNeighborsClassifier
#导入K最近邻（K-Nearest Neighbors，KNN）分类器，用于分类任务。
from sklearn.tree import DecisionTreeClassifier
#导入决策树（Decision Tree）分类器，用于分类任务。
from sklearn.ensemble import RandomForestClassifier
#：导入随机森林（Random Forest）分类器，用于分类任务。
from sklearn.model_selection import train_test_split
#导入用于将数据集拆分为训练集和测试集的函数 train_test_split。

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

综合考虑到运行时间等因素，用于限制训练集和测试集的大小，将它们分别截取前 700 个样本。

In [ ]:
X_train = X_train[:700]
y_train = y_train[:700]
X_test = X_test[:700]
y_test = y_test[:700]

支持向量机法分类

In [ ]:
# 初始化支持向量机分类器，并设置probability=True以便启用概率估计
svc = svm.SVC(probability=True)
# 使用训练集X_train和y_train训练SVM模型
svc.fit(X_train, y_train)
# 计算SVM模型在测试集X_test上的准确性，并输出
print(f"SUPPORT VECTOR MACHINE ACCURACY: {svc.score(X_test, y_test)}")

### 支持向量机（SVM）模型在测试集上的性能分析

#### 准确性
- **值**：96.43%
- **含义**：这意味着模型在测试集上对输入数据进行分类时，96.43%的样本被正确分类。

### 模型性能分析

#### 高准确性
- **表现**：SVM模型在测试集上表现出了较高的准确性，说明模型能够很好地学习训练数据的特征，并且能够有效地应用到新的测试数据上。

#### 模型性能良好
- **结论**：96.43%的准确性表明SVM模型在处理当前数据集时具有较强的预测能力，适合用于类似的分类任务。

### 进一步优化
- **建议**：虽然当前结果已经较好，但仍可以通过以下方法进一步提高模型性能：
  - **调节模型参数**：例如调整核函数类型、正则化参数等。
  - **增加数据量**：获取更多的训练数据以提升模型的泛化能力。
  - **使用数据增强**：通过数据增强技术来增加训练数据的多样性，减少过拟合现象。


使用训练好的支持向量机（SVM）模型 `svc` 对测试集数据 `X_test` 进行预测，返回预测结果。

In [ ]:
svc.predict(X_test)

模型预测输出的结果是一个数组，其中每个元素代表相应测试样本的预测标签。

KNN法分类

In [ ]:
# 创建KNN模型实例
knn = KNeighborsClassifier()
# 使用训练数据训练KNN模型
knn.fit(X_train,y_train)
# 输出KNN模型在测试集上的准确率
print(f" KNN  ACCURACY ：{knn.score(X_test,y_test)}")

这表明 KNN 模型在测试集上的准确率为 95.71%。该结果显示 KNN 模型在测试集上的表现较好，能够较为准确地分类大多数样本。与之前的 SVM 模型比较，SVM 的准确率为 96.43%，略高于 KNN 模型的表现。

 使用训练好的 KNN 模型对测试集 X_test 进行预测，返回预测标签。

In [ ]:
knn.predict(X_test)

预测结果概述：

数组中的每个值对应于一个测试样本的预测标签。
预测标签为 0 表示模型预测该样本为正常。
预测标签为 1 表示模型预测该样本为肺结核。
样本测情况：

从图片中可以看到，预测结果包含大量的 0，这表明在测试集中有很多样本被预测为正常。
也有一些 1，表示有部分样本被预测为肺结核。

决策树法分类

In [ ]:
# 创建决策树分类器的实例
dt = DecisionTreeClassifier()
# 用训练数据拟合决策树模型
dt.fit(X_train,y_train)
# 在测试集上评估模型的准确率并打印结果
print(f" Decision Tree accuracy {dt.score(X_test,y_test)}")

### 准确率解释

#### 准确率（Accuracy）
- **定义**：准确率是指模型在测试集上正确分类的样本数占总样本数的比例。
- **值**：0.9214285714285714
- **含义**：表示在所有测试样本中，有 92.14% 的样本被正确分类。具体来说，如果测试集中有 100 个样本，其中大约 92 个样本被模型正确分类，其余 8 个样本被错误分类。

### 性能评价

- **总体表现**：这个准确率值表明决策树模型在当前数据集上的表现是相对不错的，能够较好地分类大部分样本。
- **改进空间**：但仍然存在约 7.86% 的样本被错误分类，提示我们模型还有提升的空间。


 使用训练好的决策树模型对测试集 X_test 进行预测，返回预测标签。

In [ ]:
dt.predict(X_test)

从数组中可以看出，模型对大多数样本的预测是准确的，但具体的准确率和F1分数等评估指标已经在之前的分析中给出：

决策树模型在测试集上的准确率为 0.9214285714285714。这意味着该模型在测试集上有大约92.14%的预测是正确的。

随机森林法分类

In [ ]:
# 创建随机森林分类器的实例
rf = RandomForestClassifier()
# 使用训练数据训练模型
rf.fit(X_train,y_train)
print(f"Random Forest accuracy : {rf.score(X_test,y_test)}")

### 结果分析

#### 准确率
- **值**：0.9628571428571429
- **含义**：即约96.29%。这一结果表明，模型在绝大多数情况下能够正确预测测试样本的类别。

### 模型性能
- **表现**：随机森林模型在这次任务中表现优异，准确率接近96.29%。这表明该模型能够有效地处理分类任务，具有较高的预测准确性。

### 模型优势
- **集成学习**：随机森林通过集成多棵决策树的预测结果，减少了单个模型可能出现的过拟合问题，提高了模型的泛化能力。
- **鲁棒性**：这种集成方法使得随机森林在处理不同类型的数据集时表现更加稳定和可靠。


 使用训练好的随机森林模型对测试集 X_test 进行预测，返回预测标签。

In [ ]:
rf.predict(X_test)

数组中的每个值表示模型对相应样本的预测类别。这里，0 和 1 分别代表两种不同的分类结果（例如，0 代表正常，1 代表肺结核）。
输出结果中的 0 和 1 的分布展示了模型对测试集样本的分类情况。根据模型的准确率（96.29%），大部分预测是准确的。

评估测试结果

定义一个评估模型的函数，评估和比较四种机器学习算法（支持向量机SVM、K近邻KNN、决策树Decision Tree和随机森林Random Forest）的分类性能

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate_model(model, X_test, y_test):
       # 使用模型预测测试集
    y_pred = model.predict(X_test)
    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred)
    # 计算加权平均的F1得分
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, f1
#分别评估四种模型
accuracy_svm, f1_svm = evaluate_model(svc, X_test, y_test)
accuracy_knn, f1_knn = evaluate_model(knn, X_test, y_test)
accuracy_dt, f1_dt = evaluate_model(dt, X_test, y_test)
accuracy_rf, f1_rf = evaluate_model(rf, X_test, y_test)
# 输出各个模型的评估结果
print("SVM Accuracy:", accuracy_svm)
print("SVM F1 Score:", f1_svm)
print("KNN Accuracy:", accuracy_knn)
print("KNN F1 Score:", f1_knn)
print("Decision Tree Accuracy:", accuracy_dt)
print("Decision Tree F1 Score:", f1_dt)
print("Random Tree Accuracy:", accuracy_rf)
print("Random Tree F1 Score:", f1_rf)

准确率和F1得分最高的模型：随机森林模型（Random Forest）和支持向量机（SVM）在测试集上的表现最好，两者的准确率和F1得分都达到了0.9643左右，表明这两种模型在分类任务中的表现非常优越。

KNN模型表现较好：K近邻（KNN）模型的表现次之，准确率为0.9371，F1得分为0.9394，仍然表现出良好的分类能。

决策树模型表现相对较差：决策树（Decision Tree）模型的表现略逊色，准确率为0.9214，F1得分为0.9213，尽管仍然较高，但相比其他三种模型略不足。

综合评价
在四种模型中，随机森林模型和支持向量机模型表现最佳。K近邻模型次之，而决策树模型表现相对较差。根据这些评估结果，可以选择随机森林模型或支持向量机模型作为最终的分类模型，以实现最佳的分类效果。

这段代码绘制每个模型的混淆矩阵热图，通过颜色深浅表示正确分类和错误分类的数量。通过绘制每个模型的混淆矩阵，能够直观地比较各个模型在分类任务中的表现，识别出分类准确和错误分类的具体情况。

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 对所有模型进行预测
svc_pred = svc.predict(X_test)
knn_pred = knn.predict(X_test)
dt_pred = dt.predict(X_test)
rf_pred=rf.predict(X_test)

# 为每个模型创建混淆矩阵
svc_cm = confusion_matrix(y_test, svc_pred)
knn_cm = confusion_matrix(y_test, knn_pred)
dt_cm = confusion_matrix(y_test, dt_pred)
rf_cm=confusion_matrix(y_test, rf_pred)

# 绘制混淆矩阵
plt.figure(figsize=(18, 6))
plt.subplot(1, 4, 1)
sns.heatmap(svc_cm, annot=True, fmt='d', cmap='Blues', xticklabels=Categories, yticklabels=Categories)
plt.title('SVM ')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 4, 2)
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Blues', xticklabels=Categories, yticklabels=Categories)
plt.title('KNN ')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 4, 3)
sns.heatmap(dt_cm, annot=True, fmt='d', cmap='Blues', xticklabels=Categories, yticklabels=Categories)
plt.title('DF')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 4, 4)
sns.heatmap(rf_cm, annot=True, fmt='d', cmap='Blues', xticklabels=Categories, yticklabels=Categories)
plt.title('RF')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()

### 模型比较与分析

#### 准确率与精确率表现
- **最佳表现**：随机森林在准确率和精确率上均表现最佳。
- **次佳表现**：SVM次之。
- **相对较差**：KNN和决策树则表现相对较差。

#### 错误分类分析

**正常类别的错误分类**
- **SVM和随机森林**：错误分类最少，均为6。
- **KNN和决策树**：错误分类较多，分别为8和24。

**肺结核类别的错误分类**
- **KNN**：错误分类最多，有39个错误分类，表现较差。
- **决策树和SVM**：错误分类情况相同，均为32。

#### 总体表现
- **随机森林**：总体表现最好，在正常类别和肺结核类别的分类都相对准确。
- **SVM**：在正常类别上的表现和随机森林相当，但在肺结核分类上稍差。
- **决策树和KNN**：表现相对较差，特别是在肺结核分类上，KNN的错误分类数较多。

### 结论
综上所述，四种分类算法中：
- **最佳选择**：随机森林的分类效果最佳，适用于需要高分类准确性的应用场景。
- **次佳选择**：SVM表现稍逊于随机森林，但仍具备较好的分类能力。
- **相对较差**：决策树和KNN的分类效果较差，特别是在处理肺结核分类任务时表现不佳。

对于需要高分类准确性的应用场景，可以优先考虑使用随机森林算法。



计算模型的ROC曲线及其对应的AUC值，并绘制出ROC曲线，以评估模型的分类性能

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 为SVM获取预测概率
svc_probs = svc.predict_proba(X_test)[:, 1]

# 为KNN获取预测概率
knn_probs = knn.predict_proba(X_test)[:, 1]

# 决策树和随机森林通常返回概率，所以可以直接使用predict_proba
dt_probs = dt.predict_proba(X_test)[:, 1]
rf_probs = rf.predict_proba(X_test)[:, 1]

# 计算ROC曲线的FPR和TPR
svc_fpr, svc_tpr, _ = roc_curve(y_test, svc_probs)
knn_fpr, knn_tpr, _ = roc_curve(y_test, knn_probs)
dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_probs)
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probs)

# 计算AUC
svc_auc = auc(svc_fpr, svc_tpr)
knn_auc = auc(knn_fpr, knn_tpr)
dt_auc = auc(dt_fpr, dt_tpr)
rf_auc = auc(rf_fpr, rf_tpr)
# 绘制ROC曲线
plt.figure(figsize=(10, 8))

plt.plot(svc_fpr, svc_tpr, label=f'SVM (AUC = {svc_auc:.2f})')
plt.plot(knn_fpr, knn_tpr, label=f'KNN (AUC = {knn_auc:.2f})')
plt.plot(dt_fpr, dt_tpr, label=f'Decision Tree (AUC = {dt_auc:.2f})')
plt.plot(rf_fpr, rf_tpr, label=f'Random Forest (AUC = {rf_auc:.2f})')

# 绘制随机分类器的ROC曲线
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

### 综合比较

#### SVM
- **AUC值**：最高（0.99）
- **ROC曲线**：最接近理想状态，分类性能最佳。
- **适用场景**：适用于高要求的分类任务，性能最佳。

#### 随机森林
- **AUC值**：次高（0.98）
- **分类性能**：同样优异，仅次于SVM。
- **适用场景**：综合性能优异，适用于大多数分类任务。

#### KNN
- **AUC值**：0.93
- **分类性能**：整体表现良好，但在某些阈值下的表现不如SVM和随机森林。
- **适用场景**：性能较好，但在处理复杂分类任务时可能不如前两者稳定。

#### 决策树
- **AUC值**：最低（0.90）
- **分类性能**：相对较弱，需进一步优化。
- **适用场景**：性能较弱，需进一步优化和改进。

### 结论
通过上述分析，可以确定：
- **最佳选择**：SVM和随机森林是本次分类任务中的最佳选择，分别在准确性和稳定性上表现出色。
  - **SVM**：适用于高要求的分类任务，性能最佳。
  - **随机森林**：综合性能优异，适用于大多数分类任务。
- **次佳选择**：KNN，性能较好，但在处理复杂分类任务时可能不如前两者稳定。
- **进一步优化**：决策树，需进一步优化和改进以提升分类性能。

### 综合表现总结
- **SVM**：适用于高要求的分类任务，性能最佳。
- **随机森林**：综合性能优异，适用于大多数分类任务。
- **KNN**：性能较好，但在复杂分类任务上略逊一筹。
- **决策树**：需进一步优化，当前分类性能最弱。

在实际应用中，可以优先考虑SVM和随机森林算法，以实现更高的分类准确性和稳定性。


--------戴政部分2结束------

--------陶四能部分开始------

In [ ]:
import torch  # PyTorch库，用于深度学习的构建和训练
import numpy as np  # NumPy库，用于数组和矩阵运算，提供了高效的数值计算工具
from pathlib import Path  # Pathlib库，用于处理文件和目录路径
import cv2  # OpenCV库，用于计算机视觉任务，如图像处理和视频处理
import math  # 数学库，提供基础的数学函数和常量
import pandas as pd  # Pandas库，用于数据分析和数据处理
from skimage.transform import resize  # Scikit-Image库中的resize函数，用于图像缩放
from skimage.io import imread  # Scikit-Image库中的imread函数，用于读取图像文件
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report  # Scikit-Learn库，用于计算评价指标和可视化混淆矩阵
from collections.abc import Iterable  # Collections模块中的Iterable类，用于检查对象是否为可迭代对象
import matplotlib.pyplot as plt  # Matplotlib库中的pyplot模块，用于数据可视化
import os  # OS模块，用于操作系统相关功能，如文件和目录操作
import timm  # PyTorch Image Models库，提供了大量预训练的视觉模型
import torchvision  # Torchvision库，提供了用于计算机视觉的工具和预训练模型

In [ ]:
# 定义批量大小为30
batch_size = 30
# 定义图像块的大小为16
patch_size = 16
# 定义输入图像的大小为224
img_size = 224
# 计算图像中块的数量（224x224的图像分成16x16的块）
num_patches = (img_size // patch_size) ** 2
# 定义位置嵌入的维度为768
p_dim = 768
# 定义自注意力头的数量为12
heads_att = 12
# 定义编码器的数量为12
num_encoder = 12

数据处理阶段
读取图像文件：使用 glob 方法从指定目录（"TB_Chest_Radiography_Database/Normal/" 和 "TB_Chest_Radiography_Database/Tuberculosis/"）中读取所有 PNG 格式的图像文件，并将这些文件路径转换为字符串格式存储。
排序图像文件：通过 sorted() 函数对读取到的图像路径进行排序，确保数据的顺序一致，这对后续数据处理和模型训练的稳定性很重要。
创建标签：为每个图像分配一个标签。对于正常的X光图像（Normal_images），分配标签 0；对于结核病的X光图像（TB_images），分配标签 1。这样做是为了在后续的机器学习模型中使用这些标签来训练识别正常和结核病图像的分类器。

In [ ]:
target_class={0:"Normal",1:"Tuberculosis"}
Normal_dir=Path("TB_Chest_Radiography_Database/Normal/")
TB_dir=Path("TB_Chest_Radiography_Database/Tuberculosis/")

使用glob方法匹配所有目标格式的文件并生成文件路径列表；通过 map 函数将路径对象转换为字符串，并使用 sorted 函数

对路径列表进行排序，确保文件按字母顺序排列。

为不同类别图像分配标签。

In [ ]:
Normal_images:list=sorted(list(map(str, list(Normal_dir.glob("*.png")))))
TB_images:list=sorted(list(map(str, list(TB_dir.glob("*.png")))))
Normal_labels:list=[0]*len(Normal_images)
TB_labels:list=[1]*len(TB_images)

In [ ]:
images=np.array(Normal_images+TB_images)
labels=np.array(Normal_labels+TB_labels)
images.shape, labels.shape

运行结果显示 images 和 labels 数组的形状均为 (4200,)
images 数组包含4200个元素，每个元素是一个图像文件的路径；labels 数组也包含4200个元素，每个元素是对应图像的标签。
标签数组与图像路径数组的长度一致，确保每个图像有一个对应的标签。
结果表明，正常胸部X光图像和结核病胸部X光图像的总数（即数据集）加起来是4200张。
此运行结果验证了数据集中的图像和标签已正确读取并合并，每个图像都有一个对应的标签，并确保数据集大小和标签分配正确。

下面将图像路径和标签数据集分成训练集、验证集和测试集

In [ ]:
# 导入train_test_split函数，用于划分数据集
from sklearn.model_selection import train_test_split
# 第一次划分：将数据集分成训练集(80%)和临时验证集(20%)
x_train, x_valid, y_train, y_valid = train_test_split(images,labels,test_size=0.2,random_state=42)
# 第二次划分：将临时验证集分成最终验证集(10%)和测试集(10%)
x_valid,x_test,y_valid,y_test=train_test_split(x_valid,y_valid,test_size=0.5,random_state=42)

(x_train.shape,x_valid.shape,x_test.shape)

训练集 (x_train): 3360 个样本，用于训练模型。
验证集 (x_valid): 420 个样本，用于训练期间的模型性能评估和参数调优
测试集 (x_test): 420 个样本，用于最终模型性能评
结果表明数据集已经按预期比例正确划分，每个子集的样本数与预期相符。

定义函数对输入的图像进行预处理，包括灰度转换、对比度增强、模糊处理、调整大小和颜色空间转换，并最终将图像的通道顺序调整为深度学习模型所需的格式
其中利用自适应直方图均衡化 (CLAHE) 增强局部对比度，突出图像细节，防止过度增强；利用高斯模糊平滑图像，减少噪声和细节。
将两者结合，改善图像质量，使后续处理和分析更有效。

In [ ]:
def image_preprocessing(path):
    img=cv2.imread(path)
     # 将图像转换为灰度图像
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 创建自适应直方图均衡化对象，clipLimit为对比度限制
    clahe=cv2.createCLAHE(clipLimit=2)
    # 应用自适应直方图均衡化
    img=clahe.apply(img)
    # 使用高斯模糊处理图像，内核大小为5x5
    img=cv2.GaussianBlur(img,(5,5),0,borderType=cv2.BORDER_CONSTANT)
    # 调整图像大小
    img=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_LINEAR)
    # 将图像从灰度转换为RGB图像
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    # 将图像的轴从 (height, width, channels) 转换为 (channels, height, width)
    img=np.moveaxis(img,-1,0)
    return img

对训练集、验证集和测试集中的所有图像进行预处理，并将它们转换为浮点型数组
使用 map 函数对练集、验证集和测试集中的每个元素（图像路径）应用 image_preprocessing 函数，
将处理后的图像列表转换为 NumPy 数组。
使用 astype(np.float32) 将数组的数据类型转换为 32 位浮点型，以提高计算精度和兼容性。

In [ ]:
x_train=np.array(list(map(image_preprocessing,x_train))).astype(np.float32)
x_valid=np.array(list(map(image_preprocessing,x_valid))).astype(np.float32)
x_test=np.array(list(map(image_preprocessing,x_test))).astype(np.float32)
x_train.shape,x_valid.shape,x_test.shape

将预处理后的图像数据和标签转换为 PyTorch 张量，并移动到指定的设备（CPU 或 GPU）上，以便进行后续的模型训练和评估

In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
x_train=torch.from_numpy(x_train).to(device)
x_valid=torch.from_numpy(x_valid).to(device)
x_test=torch.from_numpy(x_test).to(device)
y_train=torch.from_numpy(y_train).to(device)
y_valid=torch.from_numpy(y_valid).to(device)
y_test=torch.from_numpy(y_test).to(device)

定义一个标准化处理类StandardScaler，用于对 PyTorch 张量进行标准化操作

In [ ]:
class StandardScaler():
    def __init__(self) -> None:
        # 初始化均值和标准差
        self.mean = None
        self.std = None

    def fit(self, tensor: torch.Tensor) -> None:
        # 计算并存储张量的均值和标准差
        self.mean = tensor.mean((0, 2, 3), keepdim=True)
        self.std = tensor.std((0, 2, 3), keepdim=True)

    def transform(self, tensor: torch.Tensor) -> torch.Tensor:
        # 使用计算得出的均值和标准差对张量进行标准化
        scaled = (tensor - self.mean) / (self.std + 1e-5)
        return scaled

    def fit_transform(self, tensor: torch.Tensor) -> torch.Tensor:
        # 计算均值和标准差，并对张量进行标准化
        self.fit(tensor=tensor)
        scaled = self.transform(tensor=tensor)
        return scaled

接着使用StandardScaler类对训练集、验证集和测试集的图像数据进行标准化处理

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_valid=scaler.transform(x_valid)

从 torch.utils.data 模块导入 TensorDataset 和 DataLoader 两个类，以便将预处理后的数据集转换为 PyTorch 的数据集对象，并创建数据加载器以便于批量处理数据。
TensorDataset 是一个将张量数据包装成数据集对象的类，可以将输入数据和标签打包在一起，便于数据加载和迭代。
使用 TensorDataset 可以方便地将图像数据和标签结合起来，创建一个可迭代的数据集对象。
DataLoader 是一个用于批量加载数据的类，可以从数据集对象中按批次加载数据，并提供对数据的随机访问、打乱和并行加载等功能。
使用 DataLoader 可以方便地将数据集分成多个小批次，在训练过程中按批次加载数据，提高训练效率。

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
# 将训练集、验证集和测试集的图像数据和标签封装成 TensorDataset 对象
train_dataset=TensorDataset(x_train,y_train)
val_dataset=TensorDataset(x_valid,y_valid)
test_dataset=TensorDataset(x_test,y_test)
# 创建训练集、验证集和测试集的数据加载器，设置批量大小和是否打乱数据
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

resnext50_32x4d 是 ResNeXt 网络架构的一种变体。ResNeXt 是一种卷积神经网络（CNN）架构，由 Facebook 的研究团队提出，它结合了 ResNet 和 Inception 模型的优点。

In [ ]:
base = torchvision.models.resnext50_32x4d(num_classes=1)

定义了一个自定义神经网络模型类 ResNext，继承自 torch.nn.Module。该模型在预训练的 base 模型基础上增加了一个 sigmoid 激活函数层，用于将模型输出转化为概率值

In [ ]:
class ResNext(torch.nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base=base
        self.sigmoid=torch.nn.Sigmoid()
    def forward(self,x):
        return self.sigmoid(self.base(x))

In [ ]:
model=ResNext(base).to(device)
print(model)  # 打印模型结构。

In [ ]:
# 计算并打印模型的总参数数量
total_params = sum(p.numel() for p in model.parameters())
total_params

模型的参数量为22981953

设置了训练模型的超参数和优化策略，包括训练的轮数、损失函数、优化器以及学习率调度器。它们将用于控制模型在训练过程中的行为。
余弦退火学习率调度器会在训练过程中逐步调整学习率，以提高训练效果和模型性能。

In [ ]:
# 设置训练轮数
epoch=100
# 定义损失函数，使用二分类交叉熵损失函数
loss_fn=torch.nn.BCELoss()
# 定义优化器，使用随机梯度下降（SGD），学习率为 1e-3，动量为 0.9
opt=torch.optim.SGD(model.parameters(),lr=1e-3,momentum=0.9)
# 定义学习率调度器，使用余弦退火学习率调度器
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=opt,T_max=epoch,verbose=True)

In [ ]:
best_loss = float('inf')  # 初始化最优损失为正无穷
best_model_weights = None  # 初始化最优模型权重
patience = 10  # 早停策略的耐心值
train_losses = []  # 用于存储每个epoch的训练损失
val_losses = []  # 用于存储每个epoch的验证损失

for i in range(epoch):
    print(f"Epoch: {i+1}\n")
    model.train()  # 设置模型为训练模式
    train_loss = 0  # 初始化训练损失
    train_batches = len(train_loader)  # 获取训练批次数量
    
    # 训练循环
    for batch, (x, y) in enumerate(train_loader):
        pred = model(x).squeeze(-1)  # 模型预测
        loss = loss_fn(pred, y.float())  # 计算损失
        train_loss += loss.item()  # 累积损失
        loss.backward()  # 反向传播
        opt.step()  # 更新参数
        opt.zero_grad()  # 清空梯度
    
    train_loss /= train_batches  # 计算平均训练损失
    train_losses.append(train_loss)  # 记录训练损失
    print(f" \n train loss: {train_loss:>8f} \n")
    
    model.eval()  # 设置模型为评估模式
    val_loss = 0  
    val_batches = len(val_loader)  # 获取验证批次数量
    
    # 验证循环
    with torch.no_grad():
        for x, y in val_loader:
            pred = model(x).squeeze(-1)  # 模型预测
            val_loss += loss_fn(pred, y.float()).item()  # 累积损失
    
    val_loss /= val_batches  # 计算平均验证损失
    val_losses.append(val_loss)  # 记录验证损失
    print(f" \n test loss: {val_loss:>8f} \n")
    
    # 检查是否为最优模型
    if val_loss < best_loss:
        best_loss = val_loss  # 更新最优损失
        best_model_weights = model.state_dict()  # 更新最优模型权重
        patience = 10  #重置
    else:
        patience -= 1  
        if patience == 0:  # 停止训练
            break
    
    scheduler.step()  # 更新学习率

# 加载最优模型权重
model.load_state_dict(best_model_weights)
print(f'best loss:', best_loss)
print("ResNeXt训练结束--林家苏")

可视化观察模型在训练集和验证集上的损失变化

In [ ]:
# 设置图形大小，宽度为 20 英寸，高度为 7 英寸
plt.figure(figsize=(20, 7))
# 绘制训练损失曲线
# x 轴为训练的 epoch 数，y 轴为训练损失
print("---ResneXt损失曲线--许海天---")
plt.plot(range(len(train_losses)), train_losses)
# 绘制验证损失曲线
# x 轴为训练的 epoch 数，y 轴为验证损失
plt.plot(range(len(val_losses)), val_losses)
# 添加图例，分别标记训练损失和验证损失曲线
plt.legend(['loss', 'val_loss'], loc='upper right')
# 显示绘制的图形
plt.show()


### 结果分析

#### 损失变化趋势
- **初始阶段（0-2个epoch）**：验证损失波动较大，达到一个峰值后迅速下降。
- **中期阶段（2-10个epoch）**：验证损失波动逐渐减小，但仍存在起伏。
- **后期阶段（10-20个epoch）**：训练损失和验证损失均趋于稳定，验证损失略高于训练损失。

#### 结论分析

**模型收敛性**：
- 训练损失持续下降，并在训练后期趋于稳定，表明模型在训练集上具有良好的收敛性。
- 验证损失在初始阶段波动较大，但随后逐渐减小并趋于稳定，说明模型在验证集上的表现逐渐稳定。

**验证损失波动**：
- 初始阶段验证损失的波动较大，可能是由于模型在早期对数据的学习尚不稳定，或数据集存在噪声。
- 中期阶段验证损失仍有起伏，但总体趋势在下降，表明模型在逐渐适应验证集的数据。

**模型性能**：
- ResNeXt模型最终训练损失和验证损失都达到较低值，表明模型在训练集和验证集上均有较好的表现。
- 验证损失略高于训练损失，表明模型有一定的泛化能力，但仍需关注验证损失的波动情型在实际应用中的稳定性。








在模型评估模式下，计算验证集和测试集的预测结果，并根据这些预测结果计算召回率和准确率

In [ ]:
#评估模型在验证集上的性能
# 切换模型到评估模式
model.eval()
# 初始化用于存储验证集预测结果的列表
val_preds=[]
# 禁用梯度计算以节省内存和加速计算
with torch.no_grad():
    # 遍历验证数据加载器中的每一个批次
    for x,y in val_loader:
         # 使用模型进行预测，并移除最后一维（将输出的形状从 [batch_size, 1] 变为 [batch_size]）
        pred=model(x).squeeze(-1)
        # 将预测结果从 GPU 移动到 CPU，并转换为 numpy 数组，添加到 val_preds 列表中
        val_preds.append(pred.cpu().numpy())
        # 将列表中的所有批次预测结果拼接成一个 numpy 数组
val_preds=np.concatenate(val_preds,axis=0)  
# 对预测结果进行四舍五入，以得到二分类的最终预测结果（0 或 1）
val_preds=np.round(val_preds)
# 计算验证集的召回率和准确率
recall_score(y_valid.cpu().numpy(),val_preds),accuracy_score(y_valid.cpu().numpy(),val_preds)

### 运行结果

#### 召回率：0.9295774647878324
- 这个指标表示在所有真实为正样本（1）的样本中，模型正确预测为正样本的比例。
- 即模型能够正确识别大部分的正样本，但仍有一部分正样本被错误分类。

#### 准确率：0.983333333333333
- 这个指标表示在所有样本中，模型预测正确的样本比例。
- 较高的准确率表明模型整体表现良好，绝大部分样本都被正确分类。

#### 模型性能
- **召回率和准确率都相对较高**，表明模型在验证集上具有良好的分类性能。
- 由于召回率略低于准确率，模型可能在识别某些正样本时存在一定的误差。这可能需要进一步优化模型或调整分类阈值。


In [ ]:
#评估模型在测试集上的性能
model.eval()
# 初始化存储测试集预测结果的列表
test_preds = []
# 禁用梯度计算，以减少内存消耗和计算开销
with torch.no_grad():
    # 遍历测试集数据加载器，生成预测结果
    for x, y in test_loader:
        # 模型预测，并去掉最后一个维度
        pred = model(x).squeeze(-1)
        # 将预测结果从 GPU 移动到 CPU，并转换为 numpy 数组
        test_preds.append(pred.cpu().numpy())
# 将所有批次的预测结果拼接成一个数组
test_preds = np.concatenate(test_preds, axis=0)
# 将预测结果进行四舍五入，得到二分类结果（0 或 1）
test_preds = np.round(test_preds)
#计算召回率和准确率
recall_score(y_test.cpu().numpy(),test_preds),accuracy_score(y_test.cpu().numpy(),test_preds)

### 运行结果

#### 召回率：0.9242424242424242
- 这个指标表示在所有真实为正样本（1）的样本中，模型正确预测为正样本的比例。
- 召回率略低，说明模型有一部分正样本被错误分类为负样本（0）。

#### 准确率：0.9880952380952381
- 这个指标表示在所有样本中，模型预测正确的样本比例。
- 较高的准确率表明模型整体表现良好，绝大部分样本都被正确分类。

#### 模型性能
- **虽然准确率较高，但召回率略低**，表明模型在识别正样本时存在一些误差。
- 这可能会在某些应用场景中（例如需要更高召回率的任务）带来问题。

定义函数计算测试结果

In [ ]:
def evaluation_parametrics(name,y_val, y_pred):
    print("\n{}\n".format(name))  
    # 打印分类报告，包括精确度、召回率、F1分数和支持度
    print(classification_report(y_val, y_pred)) 

定义画混淆矩阵的函数

In [ ]:
def eva_ConfusionMatrixDisplay(name,y_val, y_pred):
    cm_test = confusion_matrix(y_val, y_pred)
    t1 = ConfusionMatrixDisplay(cm_test)    
    print("\n{}\n".format(name))
    t1.plot()

调用 evaluation_parametrics 函数对验证集和测试集的预测结果进行评估，生成分类报告

In [ ]:
evaluation_parametrics("ResneXt50 验证集结果--许海天", y_valid.cpu().numpy(), val_preds)
evaluation_parametrics("ResneXt50 测试集结果--许海天", y_test.cpu().numpy(), test_preds)

### 验证集结果

#### Precision（精确率）
- **类别0（正常）**：验证集的精确率为0.99，说明模型在预测正常样本时的准确率非常高。
- **类别1（肺结核）**：验证集的精确率为0.97，表明模型在预测肺结核样本时的误报率较低。

#### Recall（召回率）
- **类别0（正常）**：验证集的召回率为0.99，说明模型能够正确识别出绝大多数正常样本。
- **类别1（肺结核）**：验证集的召回率为0.93，表明模型在识别肺结核样本时有一定漏报。

#### F1-score
- **类别0（正常）**：验证集的F1-score为0.99，表明模型在正常样本上的综合表现非常好。
- **类别1（肺结核）**：验证集的F1-score为0.95，表明模型在肺结核样本上的综合表现较好。

#### Accuracy（准确率）
- 验证集的准确率为0.98，说明模型在整体分类任务中的表现非常出色。

#### Macro avg 和 weighted avg
- **Macro avg**：0.98，说明模型在各类别上的表现均衡。
- **Weighted avg**：0.98，进一步证明模型的整体表现非常优秀。

### 测试结果

#### Precision（精确率）
- **类别0（正常）**：测试集的精确率为0.99，说明模型在预测正常样本时的准确率非常高。
- **类别1（肺结核）**：测试集的精确率为1.00，表明模型在预测肺结核样本时几乎没有误报。

#### Recall（召回率）
- **类别0（正常）**：测试集的召回率为1.00，说明模型能够正确识别出所有正常样本。
- **类别1（肺结核）**：测试集的召回率为0.92，表明模型在识别肺结核样本时有一定漏报，但整体表现仍然良好。

#### F1-score
- **类别0（正常）**：测试集的F1-score为0.99，表明模型在正常样本上的综合表现非常好。
- **类别1（肺结核）**：测试集的F1-score为0.96，表明模型在肺结核样本上的综合表现较好。

#### Accuracy（准确率）
- 测试集的准确率为0.98，说明模型在实际应用中的分类效果非常好。

#### Macro avg 和 weighted avg
- **Macro avg**：0.98，说明模型在各类别上的表现均衡。
- **Weighted avg**：0.98，进一步证明模型的整体表现非常优秀。

### 结论
综上所述，ResNeXt50模型在验证集和测试集上的表现非常出色。模型能够很好地识别出正常和肺结核样本，具有高精确率、高召回率和高F1-score。同时，测试集的准确率为0.98，表明模型在实际应用中的分类效果也非常好。这表明ResNeXt50模型能够有效地用于肺结核的检测和分类。
分效果也非常好。这表明ResNeXt50模型能够有效地用于肺结核的检测和分类。


调用 eva_ConfusionMatrixDisplay 函数对验证集和测试集的预测结果进行评估，生成混淆矩阵

In [ ]:
eva_ConfusionMatrixDisplay("ResNeXt50 验证集混淆矩阵--许海天", y_valid.cpu().numpy(), val_preds)

In [ ]:
eva_ConfusionMatrixDisplay("ResNeXt50 测试集混淆矩阵--许海天", y_test.cpu().numpy(), test_preds)

### 混淆矩阵解读

#### ResNeXt50 模型在验证集上的混淆矩阵
- 真阳性 (True Positive)：347
- 假阳性 (False Positive)：2
- 假阴性 (False Negative)：5
- 真阴性 (True Negative)：66

#### ResNeXt50 模型在测试集上的混淆矩阵
- 真阳性 (True Positive)：354
- 假阳性 (False Positive)：0
- 假阴性 (False Negative)：5
- 真阴性 (True Negative)：61

### 分析

#### 1. 准确率和召回率

**验证集**：
- 模型在验证集上的表现非常好，大部分样本都被正确分类，假阳性和假阴性都较少。
- 准确率和召回率都很高，说明模型在验证集上的学习效果良好。

**测试集**：
- 模型在测试集上的表现也非常好，准确率依然保持在高水平，但假阴性的数量略有增加。
- 虽然假阳性为零，但假阴性的存在表明模型可能存在一定的过拟合现象。

#### 2. 泛化能力
- 模型在未见过的数据上的表现也非常优秀，说明ResNeXt50模型具有良好的泛化能力。
- 但是，测试集上假阴性的增加表明模型可能需要进一步优化。

### 结论
- ResNeXt50模型在分类任务中的表现非常优秀，适用于肺结核的诊断任务。
- 尽管模型在测试集上有些假阴性，但总体而言，模型的准确率和召回率都很高，具有良好的应用前景。
- 通过进一步优化模型，可以进一步提升其在实际应用中的表现，提高在不同数据集上的稳定性。现。一步优化，以提高在不同数据集上的稳定性。

计算模型的ROC曲线及其对应的AUC值，并绘制出ROC曲线，以评估模型的分类性能

In [ ]:
from sklearn.metrics import roc_curve, auc
# 计算ROC曲线的FPR和TPR
fpr_val, tpr_val, _ = roc_curve(y_valid.cpu().numpy(), val_preds)
fpr_test, tpr_test, _ = roc_curve(y_test.cpu().numpy(), test_preds)

# 计算AUC
roc_auc_val = auc(fpr_val, tpr_val)
roc_auc_test = auc(fpr_test, tpr_test)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr_val, tpr_val, label=f'Validation ROC curve (AUC = {roc_auc_val:.2f})')
plt.plot(fpr_test, tpr_test, label=f'Test ROC curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # 随机分类器的ROC曲线
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()



#### 图片内容
- **蓝色曲线**：验证集上的ROC曲线
- **橙色曲线**：测试集上的ROC曲线
- **AUC值**：验证集和测试集的AUC均为0.96

#### ROC曲线和AUC值解读

**ROC曲线**：
- ROC曲线表示模型的分类性能，曲线下方的面积越大，模型的性能越好。
- 曲线从左下角（0,0）到右上角（1,1）的路径表示不同的阈值下，模型的真阳性率（True Positive Rate, TPR）和假阳性率（False Positive Rate, FPR）的变化情况。

**AUC值**：
- AUC（Area Under the Curve）值表示ROC曲线下的面积。
- AUC值范围在0.5到1之间，值越接近1，模型的性能越好。
- 验证集和测试集的AUC值均为0.96，表明模型在这两个数据集上的分类性能都非常优秀。

#### 模型性能分析

**高AUC值**：
- 验证集和测试集上的AUC值均为0.96，表明模型在这两个数据集上都有很高的区分能力。
- 高AUC值意味着模型在各种阈值下都能保持较高的真阳性率和较低的假阳性率。

**一致的表现**：
- 验证集和测试集的ROC曲线非常接近，表明模型的性能在不同数据集上都非常一致。
- 这种一致性说明模型的泛化能力较好，能够在未见过的数据上保持良好的分类性能。

VIT部分

In [ ]:
def extract_image_patches(x: torch.Tensor, kernel: int, stride: int = 1, dilation: int = 1):
    # 获取输入张量的形状
    b, c, h, w = x.shape
    # 计算输出张量的高度和宽度
    h2 = math.ceil(h / stride)
    w2 = math.ceil(w / stride)
    # 计算需要填充的行和列
    pad_row = (h2 - 1) * stride + (kernel - 1) * dilation + 1 - h
    pad_col = (w2 - 1) * stride + (kernel - 1) * dilation + 1 - w
    # 填充输入张量
    x = torch.nn.functional.pad(x, (pad_row // 2, pad_row - pad_row // 2, pad_col // 2, pad_col - pad_col // 2))
    
    # 提取图像块
    patches = x.unfold(2, kernel, stride).unfold(3, kernel, stride)
    # 调整张量维度顺序并展平图像块
    patches = patches.permute(0, 2, 3, 1, 4, 5).contiguous()
    patches = patches.view(*patches.size()[:3], -1)
    
    # 返回展平后的图像块
    return patches.view(b, -1, patches.shape[-1])

定义get_positional_embeddings为用于生成位置嵌入的函数。
通过使用正弦和余弦函数，这些嵌入向量在不同维度上具有不同的频率，使模型能够更好地捕捉序列中的顺序关系。

In [ ]:
def get_positional_embeddings(sequence_length: int, d: int):
    # 初始化结果张量，大小为 (sequence_length, d)，初始值为 1
    result = torch.ones(sequence_length, d)
    
    # 遍历序列长度和维度，计算位置嵌入
    for i in range(sequence_length):
        for j in range(d):
            # 计算位置嵌入
            result[i][j] = (
                np.sin(i / (10000 ** (j / d)))
                if j % 2 == 0
                else np.cos(i / (10000 ** ((j - 1) / d)))
            )
    return result

定义一个编码器块类，用于实现 Transformer 模型中的一个编码器层

In [ ]:
class EncoderBlock(torch.nn.Module):
    def __init__(self, projection_dim: int, num_heads: int, num_patches: int):
        super().__init__()
        # 初始化参数
        self.projection_dim = projection_dim
        self.num_heads = num_heads
        self.num_patches = num_patches
        # 定义层归一化层
        self.norm1 = torch.nn.LayerNorm(self.projection_dim)
        self.norm2 = torch.nn.LayerNorm(self.projection_dim)
        # 定义多头注意力层
        self.attention = torch.nn.MultiheadAttention(self.projection_dim, self.num_heads, batch_first=True)
        # 定义多层感知机（MLP）
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(self.projection_dim, self.projection_dim * 4),
            torch.nn.GELU(),
            torch.nn.Linear(self.projection_dim * 4, self.projection_dim),
        )
    def forward(self, x):
        # 层归一化和多头注意力机制
        x1 = self.norm1(x)
        attention = self.attention(x1, x1, x1)[0]
        x2 = attention + x
        # 第二次层归一化和MLP
        x3 = self.norm2(x2)
        x3 = self.mlp(x3)
        # 残差连接
        out = x2 + x3
        return out

定义一个视觉变换器（ViT）模型类，用于图像分类任务

In [ ]:
class ViT(torch.nn.Module):
    def __init__(self, patch_size: int, num_patches: int, projection_dim: int, num_heads: int, num_encoder: int):
        super().__init__()
        # 初始化参数
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.num_heads = num_heads
        self.num_encoder = num_encoder
        # 计算输入维度（每个图像块的大小）
        self.input_d = self.patch_size * self.patch_size * 3
        # 定义线性层，将输入图像块投影到指定的维度
        self.linear = torch.nn.Linear(self.input_d, self.projection_dim)
        # 定义位置嵌入
        self.embbeding = torch.nn.Parameter(
            get_positional_embeddings(self.num_patches, self.projection_dim)
        )
        self.embbeding.requires_grad = False
        # 定义一系列编码器块
        self.blocks = torch.nn.ModuleList([
            EncoderBlock(self.projection_dim, self.num_heads, self.num_patches) for _ in range(self.num_encoder)
        ])
        # 定义层归一化层
        self.ln_out = torch.nn.LayerNorm(self.projection_dim)
        # 定义输出层
        self.out = torch.nn.Sequential(
            torch.nn.Linear(self.projection_dim, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, images: torch.Tensor):
        batch_size = images.size()[0]
        # 提取图像块
        x = extract_image_patches(images, self.patch_size, self.patch_size)
        # 投影图像块到指定的维度
        x = self.linear(x)
        # 添加位置嵌入
        pos_embed = self.embbeding.repeat(batch_size, 1, 1)
        encoded = x + pos_embed
        # 通过编码器块
        for block in self.blocks:
            encoded = block(encoded)
        # 平均池化
        rep = encoded.mean(dim=1)
        rep = self.ln_out(rep)
        # 通过输出层
        output = self.out(rep)
        return output

In [ ]:
model=ViT(patch_size=patch_size,
          num_patches=num_patches,
          projection_dim=p_dim,
          num_heads=heads_att,
          num_encoder=num_encoder).to(device)
print(model)

VIT模型结构--张海峰


In [ ]:
vit_total_params = sum(p.numel() for p in model.parameters())
vit_total_params

通过计算和分析 ViT 模型的参数总数，我们可以评估模型的复杂性和计算资源需求。
ViT 模型的总参数数目为 85,797,889，表明该模型在处理和训练过程中需要大量计算资源

设置了训练模型的超参数和优化策略，包括训练的轮数、损失函数、优化器以及学习率调度器。它们将用于控制模型在训练过程中的行为。
余弦退火学习率调度器会在训练过程中逐步调整学习率，以提高训练效果和模型性能。

In [ ]:
epoch=100
loss_fn=torch.nn.BCELoss()
opt=torch.optim.SGD(model.parameters(),lr=1e-3,momentum=0.9)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=opt,T_max=epoch,verbose=True)

训练一个深度学习模型，并在训练过程中监控验证损失，使用早停机制来防止过度训练。
代码包含了训练和验证两个阶段，并使用学习率调度器来动态调整学习率。

In [ ]:
# 初始化变量
best_loss = float('inf')  # 设置最佳损失初始值为无穷大
best_model_weights = None  # 用于存储最佳模型的权重
patience = 10  # 耐心值，当验证损失不再改善时提前停止训练
train_losses = []  # 记录每个epoch的训练损失
val_losses = []  # 记录每个epoch的验证损失
for i in range(epoch):  # 迭代训练
    print(f"Epoch: {i + 1}\n")
    # 训练阶段
    model.train()  # 设置模型为训练模式
    train_loss = 0  # 初始化训练损失
    train_batches = len(train_loader)  # 获取训练数据的批次数
    for batch, (x, y) in enumerate(train_loader):  # 遍历每个训练批次
        pred = model(x).squeeze(-1)  # 进行预测，并去掉最后一个维度
        loss = loss_fn(pred, y.float())  # 计算损失
        train_loss += loss.item()  # 累积损失
        loss.backward()  # 反向传播计算梯度
        opt.step()  # 更新模型参数
        opt.zero_grad()  # 清空梯度
    train_loss /= train_batches  # 计算平均训练损失
    train_losses.append(train_loss)  # 记录训练损失
    print(f" \n train loss: {train_loss:.8f} \n")
    # 验证阶段
    model.eval()  # 设置模型为评估模式
    val_loss = 0  # 初始化验证损失
    val_batches = len(val_loader)  # 获取验证数据的批次数
    with torch.no_grad():  # 禁用梯度计算
        for x, y in val_loader:  # 遍历每个验证批次
            pred = model(x).squeeze(-1)  # 进行预测，并去掉最后一个维度
            val_loss += loss_fn(pred, y.float()).item()  # 累积损失
    val_loss /= val_batches  # 计算平均验证损失
    val_losses.append(val_loss)  # 记录验证损失
    print(f"\n test loss: {val_loss:.8f} \n")
    # 检查验证损失是否改善
    if val_loss < best_loss:
        best_loss = val_loss  # 更新最佳损失
        best_model_weights = model.state_dict()  # 保存最佳模型的权重
        patience = 10  # 重置耐心值
    else:
        patience -= 1  # 耐心值减1
        if patience == 0:  # 当耐心值为0时，停止训练
            print("Early stopping triggered")
            break
    # 学习率调度器更新
    scheduler.step()
# 加载验证损失最小的模型权重
model.load_state_dict(best_model_weights)
print(f'Best validation loss: {best_loss:.8f}')

通过 Matplotlib 库绘制训练损失和验证损失的曲线图，以可视化的方式展示模型训练过程中损失的变化趋势

可视化观察模型在训练集和验证集上的损失变化

In [ ]:
# 设置图形大小，宽度为 20 英寸，高度为 7 英寸
plt.figure(figsize=(20, 7))
# 绘制训练损失曲线
# x 轴为训练的 epoch 数，y 轴为训练损失
print("---VIT训练损失曲线--许海天---")
plt.plot(range(len(train_losses)), train_losses)
# 绘制验证损失曲线
# x 轴为训练的 epoch 数，y 轴为验证损失
plt.plot(range(len(val_losses)), val_losses)
# 添加图例，分别标记训练损失和验证损失曲线
plt.legend(['loss', 'val_loss'], loc='upper right')
# 显示绘制的图形
plt.show()

### 结果分析

#### 初始阶段
- 在训练初期（前3个epoch），训练损失和验证损失都较高，随后快速下降，表明模型在早期同样快速学习到数据的特征。

#### 中期阶段
- 训练损失持续下降，并在接近第15个epoch时达到较低值，随后继续缓慢下降，表明模型在训练集上的拟合效果逐渐提升。

#### 验证损失波动
- 验证损失在整个训练过程中波动较大，特别是在第15个epoch前后，波动明显。这可能是由于ViT模型对数据较为敏感，特别是在小数据集或存在噪声的情况下。

#### 最终结果
- 在训练结束时，训练损失较低且稳定，验证损失略高于训练损失，并且在最后几个epoch中仍存在一些波动。

### 结论
- ViT模型在训练过程中表现出良好的收敛性，训练损失显著下降。
- 验证损失的较大波动可能反映出模型对数据敏感，可能需要更多的数据或进一步的数据增强来稳定模型的泛化性能。

在模型评估模式下，计算验证集和测试集的预测结果，并根据这些预测结果计算召回率和准确率

In [ ]:
#评估验证集
# 设置模型为评估模式，关闭 dropout 和 batch normalization
model.eval()
# 初始化存储验证集预测结果的列表
val_preds = []
# 禁用梯度计算，以减少内存消耗和计算开销
with torch.no_grad():
    # 遍历验证集数据加载器，生成预测结果
    for x, y in val_loader:
        # 模型预测，并去掉最后一个维度
        pred = model(x).squeeze(-1)
        # 将预测结果从 GPU 移动到 CPU，并转换为 numpy 数组
        val_preds.append(pred.cpu().numpy())
# 将所有批次的预测结果拼接成一个数组
val_preds = np.concatenate(val_preds, axis=0)
# 将预测结果进行四舍五入，得到二分类结果（0 或 1）
val_preds = np.round(val_preds)
#计算召回率和准确率
recall_score(y_valid.cpu().numpy(),val_preds),accuracy_score(y_valid.cpu().numpy(),val_preds)

### 运行结果

#### 召回率 (Recall)
- **值**：0.9859154929577465
- **含义**：模型在验证集上的召回率为0.9859，说明模型能够正确识别出约98.59%的目标类样本。

#### 准确率 (Accuracy)
- **值**：0.9976190476190476
- **含义**：模型在验证集上的准确率为0.9976，说明模型整体分类性能非常好，能够对约99.76%的样本进行正确分类。

### 结论
- **召回率**：模型在验证集上的召回率为0.9859，说明模型能够正确识别出绝大多数的目标类样本。
- **准确率**：模型在验证集上的准确率为0.9976，表明模型在整体分类任务中表现非常出色，能够正确分类绝大多数样本。
- **总体表现**：模型在验证集上的表现非常优秀，具有很高的召回率和准确率，表明模型在识别目标类样本和整体很好的表现。








In [ ]:
#评估测试集
model.eval()
# 初始化存储测试集预测结果的列表
test_preds = []
# 禁用梯度计算，以减少内存消耗和计算开销
with torch.no_grad():
    # 遍历测试集数据加载器，生成预测结果
    for x, y in test_loader:
        # 模型预测，并去掉最后一个维度
        pred = model(x).squeeze(-1)
        # 将预测结果从 GPU 移动到 CPU，并转换为 numpy 数组
        test_preds.append(pred.cpu().numpy())
# 将所有批次的预测结果拼接成一个数组
test_preds = np.concatenate(test_preds, axis=0)
# 将预测结果进行四舍五入，得到二分类结果（0 或 1）
test_preds = np.round(test_preds)
#计算召回率和准确率
recall_score(y_test.cpu().numpy(),test_preds),accuracy_score(y_test.cpu().numpy(),test_preds)

### 运行结果

#### 召回率 (Recall)
- **值**：0.9696969696969697
- **含义**：模型在测试集上的召回率为0.9697，说明模型能够正确识别出约96.97%的目标类样本。

#### 准确率 (Accuracy)
- **值**：0.9952380952380953
- **含义**：模型在测试集上的准确率为0.9952，说明模型整体分类性能非常好，能够对约99.52%的样本进行正确分类。

### 结论
- **召回率**：模型在测试集上的召回率为0.9697，说明模型能够正确识别出绝大多数的目标类样本。
- **准确率**：模型在测试集上的准确率为0.9952，表明模型在整体分类任务中表现非常出色，能够正确分类绝大多数样本。
- **总体表现**：模型在测试集上的表现非常优秀，具有很高的召回率和准确率，表明模型在识别目标类样本和整体分类任务中都有很好的表现。


调用 evaluation_parametrics 函数对验证集和测试集的预测结果进行评估，生成分类报告

In [ ]:
evaluation_parametrics("VIT 验证集结果", y_valid.cpu().numpy(), val_preds)
evaluation_parametrics("VIT 测试集结果", y_test.cpu().numpy(), test_preds)

### 验证集结果分析

#### Precision（精确率）
- **类别0（正常）**：精确率为1.00，表明模型在预测正常样本时没有误报。
- **类别1（肺结核）**：精确率为1.00，表明模型在预测肺结核样本时没有误报。

#### Recall（召回率）
- **类别0（正常）**：召回率为1.00，表明模型能够正确识别所有正常样本。
- **类别1（肺结核）**：召回率为0.99，表明模型在识别肺结核样本时略有漏报，但整体表现非常好。

#### F1-score
- **类别0（正常）**：F1-score为1.00，表明模型在正常样本上的表现完美。
- **类别1（肺结核）**：F1-score为0.99，表明模型在肺结核样本上的综合表现非常好。

#### Accuracy（准确率）
- 验证集的准确率为1.00，表明模型在整体分类任务中的表现非常出色，几乎没有错误分类。

#### Macro avg 和 weighted avg
- 这两个指标分别是宏平均和加权平均，考虑了各类别的表现和支持度（样本数量）。验证集的macro avg和weighted avg在所有指标上的表现都为1.00，进一步说明模型的整体表现非常优秀。

### 测试结果分析

#### Precision（精确率）
- **类别0（正常）**：精确率为0.99，表明模型在预测正常样本时有极少量误报。
- **类别1（肺结核）**：精确率为1.00，表明模型在预测肺结核样本时没有误报。

#### Recall（召回率）
- **类别0（正常）**：召回率为1.00，表明模型能够正确识别所有正常样本。
- **类别1（肺结核）**：召回率为0.97，表明模型在识别肺结核样本时略有漏报，但整体表现非常好。

#### F1-score
- **类别0（正常）**：F1-score为1.00，表明模型在正常样本上的表现完美。
- **类别1（肺结核）**：F1-score为0.98，表明模型在肺结核样本上的综合表现非常好。

#### Accuracy（准确率）
- 测试集的准确率为1.00，表明模型在整体分类任务中的表现非常出色，几乎没有错误分类。

#### Macro avg 和 weighted avg
- 测试集的macro avg和weighted avg在所有指标上的表现都非常接近1.00，进一步说明模型的整体表现非常优秀。

### 结论
从结果来看，ViT模型在验证集和测试集上的表现都非常出色。模型能够很好地识别出正常和肺结核样本，具有高精确率、高召回率和高F1-score。同时，测试集的准确率为1.00，表明模型在实际应用中的分类效果也非常好。
e。同时，测试集的准确率为1.00，表明模型在实际应用中的分类效果也非常好。

调用 eva_ConfusionMatrixDisplay 函数对验证集和测试集的预测结果进行评估，生成混淆矩阵

In [ ]:
eva_ConfusionMatrixDisplay("VIT 验证集混淆矩阵", y_valid.cpu().numpy(), val_preds)

In [ ]:
eva_ConfusionMatrixDisplay("VIT 测试集混淆矩阵", y_test.cpu().numpy(), test_preds)

### 验证集结论

- **整体表现**：ViT 模型在验证集上的表现非常好，精度和召回率都很高，说明模型能够很好地学习到数据中的模式。
- **不足之处**：对于少数的肺结核病例，有1个样本被误分类为正常。这说明在处理验证数据时，模型对少数类的表现略有不足。

### 测试集结论

- **整体表现**：ViT 模型在测试集上的表现依然很好，尤其是在精度上达到了1.00，这表明模型对正常样本的预测非常准确。
- **不足之处**：对于肺结核样本，有2个样本被误分类为正常，这说明模型在处理实际数据时，对少数类的识别仍存在一些不足。

### 总结

- **总体表现**：ViT 模型在验证集和测试集上的总体表现都非常好，特别是在正常样本的分类上，几乎没有错误。
- **改进建议**：唯一的改进点是提高对少数类（肺结核样本）的识别能力，可以考虑增加数据增强或使用更加平衡的损失函数来进一步优化模型。


In [ ]:
from sklearn.metrics import roc_curve, auc
# 计算ROC曲线的FPR和TPR
fpr_val, tpr_val, _ = roc_curve(y_valid.cpu().numpy(), val_preds)
fpr_test, tpr_test, _ = roc_curve(y_test.cpu().numpy(), test_preds)

# 计算AUC
roc_auc_val = auc(fpr_val, tpr_val)
roc_auc_test = auc(fpr_test, tpr_test)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr_val, tpr_val, label=f'Validation ROC curve (AUC = {roc_auc_val:.2f})')
plt.plot(fpr_test, tpr_test, label=f'Test ROC curve (AUC = {roc_auc_test:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # 随机分类器的ROC曲线
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
print('----VIT roc曲线')
plt.legend(loc="lower right")
plt.show()

### 验证集 ROC 曲线
- **蓝色曲线**：表示验证集上的 ROC 曲线，AUC 值为 0.99。这条曲线非常接近左上角，表明模型在验证集上的分类性能非常好，几乎能够完美地区分正常样本和肺结核样本。

### 测试集 ROC 曲线
- **橙色曲线**：表示测试集上的 ROC 曲线， AUC 值为 0.98。这条曲线同样非常接近左上角，表明模型在测试集上的分类性能也非常优秀，但略低于验证集的 AUC 值。

### 结论

#### 模型性能
- **验证集和测试集 AUC 值**：ViT 模型在验证集和测试集上的 AUC 值分别为 0.99 和 0.98，表明该模型在两个数据集上的分类性能都非常优秀。AUC 值接近 1.0 表示模型具有很高的区分正负样本能力。

#### 泛化能力
- **ROC 曲线表现**：验证集和测试集上的 ROC 曲线表现非常接近，AUC 值差异很小。这表明模型在验证集上的优秀表现也能较好地泛化到未见过的测试集数据上，说明模型具有良好的泛化能力。

#### 改进空间
- **轻微过拟合**：尽管 AUC 值已经非常高，但测试集上的 AUC 值略低于验证集，说明模型在处理实际数据时可能存在一些轻微的过拟合。可以通过数据增强、正则化或调整模型结构等方法进一步优化模型，以减少这种差异。

### 总结
- **总体表现**：ViT 模型在肺结核检测任务中的表现非常优秀，特别是在正常样本和肺结核样本的区分上具有很高的准确性和可靠性。
- **进一步提升**：为了进一步提升模型的性能，可以考虑使用更多的数据增强技术和模型调优策略，以确保模型在实际应用中的鲁棒性和泛化能力。
